In [16]:
import datetime
from collections import defaultdict

import pandas as pd

# Data loading and preprocessing

In [17]:
dir_path = ""
customers = pd.read_csv(dir_path + 'olist_customers_dataset.csv')
order_items = pd.read_csv(dir_path + 'olist_order_items_dataset.csv')
orders = pd.read_csv(dir_path + 'olist_orders_dataset.csv')
products = pd.read_csv(dir_path + 'olist_products_dataset.csv')
product_category_name_translation = pd.read_csv(dir_path + 'product_category_name_translation.csv')


In [18]:
USER_1 = {"cat": set(("cama_mesa_banho", "papelaria", "fashion_calcados")),
          "loc": {"city": "sao paulo", "state": "SP"}}

USER_2 = {"cat": set(("esporte_lazer", "moveis_decoracao", "telefonia")),
          "loc": {"city": "rio de janeiro", "state": "RJ"}}

USER_3 = {"cat": set(),
          "loc": {"city": "", "state": ""}}

DATE = datetime.date(2018, 8, 27)
MONTH = 30


In [19]:
complete_df = customers.merge(
    orders[['order_id', 'customer_id', 'order_purchase_timestamp']],
    on='customer_id'
).merge(
    order_items[['product_id', 'order_id']],
    on='order_id'
).merge(
    products[['product_category_name', 'product_id']],
    on='product_id')

complete_df["date"] = complete_df.order_purchase_timestamp.apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
complete_df = complete_df.drop(['order_purchase_timestamp', 'customer_id',
                                'customer_city', 'customer_zip_code_prefix',
                                'order_id'],
                               axis=1)
complete_df = complete_df[complete_df.date < DATE]  # remove info from future

date_diff = (DATE - complete_df.date).apply(lambda x: x.days)
last_month_df = complete_df[(date_diff <= MONTH)]


In [20]:
complete_df.columns

Index(['customer_unique_id', 'customer_state', 'product_id',
       'product_category_name', 'date'],
      dtype='object')

# Functions

## Similar customer functions

In [21]:
def find_similar_customers(df, customer, n_shared_cat=2):
    """
    Returns a list of customer id-s that bought products from at least
    'n_shared_cat' categories liked by the 'customer'.
    """
    customers_cats = df.groupby('customer_unique_id').aggregate(
        {'product_category_name': set})
    
    shared_categories = customers_cats.product_category_name.apply(
        lambda x: len(x.intersection(customer['cat'])))
    shared_categories = shared_categories[shared_categories >= n_shared_cat]
    return list(shared_categories.index)


def find_products_by_customer(df, cust_id):
    """
    Returns a list of named tuples containing id of a product and its quantity 
    bought by the customer.
    """
    customer_records = df[df.customer_unique_id == cust_id]
    bought_products = customer_records.groupby('product_id').aggregate(
        count=pd.NamedAgg(column="date", aggfunc="count")
    )
    return [x for x in bought_products.itertuples()]


def find_similar_customer_products(df, customer, n_shared_cat=2, top_n=10):
    """
    Returns a list of 'top_n' products that were bought by potentially 
    similar customers. Customers are similar if they share at least 
    'n_shared_cat' categories. Products bought by a customer are an indicator
    of liked categories.  
    """
    customers_list = find_similar_customers(df, customer, n_shared_cat)
    product_dict = defaultdict(lambda: 0)
    
    for cust_id in customers_list:
        for product in find_products_by_customer(df, cust_id):
            product_dict[product.Index] += product.count

    product_list = [(prod, count) for prod, count in product_dict.items()]
    product_list.sort(key=lambda x: x[1], reverse=True)
    
    top_products = [p[0] for p in product_list]
    return top_products[:top_n]


## Finding products functions

In [22]:
def find_popular_categories(df, n=10):
    """
    Returns list of n ordered categories with biggest sale.
    """
    sale = df.groupby('product_category_name').aggregate(
        count=pd.NamedAgg(column="date", aggfunc="count")
    )
    sale = sale.sort_values(by='count', ascending=False)
    return list(sale.index[:n])


def find_bestsellers(df, category=None, state=None, top_n=10):
    """
    Returns top_n products with biggest sale.
    If 'category' provided, then returns month bestsellers from the category.
    """
    if category:
        df = df[df.product_category_name == category]
    if state:
        df = df[df.customer_state == state]

    sale = df.groupby('product_id').aggregate(
        count=pd.NamedAgg(column="date", aggfunc="count")
    )
    sale = sale.sort_values(by='count', ascending=False)
    return list(sale.index[:top_n])


def find_trending_products(df, date, period=14, compare_period=28, top_n=10):
    """
    Returns top_n products with biggest percentage increase in sale between
    two periods of time: older and younger. 
    older_period = (date - compare_period(in days)) : (date - period(in days))
    younger_period = (date - period) : date
    """
    date_diff = (date - df.date).apply(lambda x: x.days)
    old_sale = df[
        (date_diff <= compare_period) & (date_diff > period)
        ]
    old_sale_count = old_sale.groupby('product_id').aggregate(
        count_old=pd.NamedAgg(column="date", aggfunc="count")
    )

    new_sale = df[(date_diff <= period)]
    new_sale_count = new_sale.groupby('product_id').aggregate(
        count_new=pd.NamedAgg(column="date", aggfunc="count")
    )

    sales = old_sale_count.merge(new_sale_count, on='product_id')
    sales['ratio'] = sales.count_new / sales.count_old
    sales = sales.sort_values(by='ratio', ascending=False)
    sales = sales[sales.ratio > 1]
    return list(sales.index[:top_n])


def find_sezonal_products(df, date_month, top_n=10):
    """
    Returns an ordered list of products that were sold more often in the
    given month comparing to a whole year.
    """
    year_df = df[
        (df.date >= datetime.date(2017, 1, 1))
        & (df.date < datetime.date(2018, 1, 1))
        ]
    year_sale = year_df.groupby('product_id').aggregate(
        year_sale=pd.NamedAgg(column="date", aggfunc="count"))

    year_sale = year_sale[year_sale > 1]
    year_mean = year_sale / 12
    year_mean.columns = ["year_mean"]

    month_df = df[(df.date.apply(lambda x: x.month) == date_month)]
    month_sale = month_df.groupby('product_id').aggregate(
        month_sale=pd.NamedAgg(column="date", aggfunc="count"))
    month_sale = month_sale.merge(year_mean, on="product_id")
    month_sale["ratio"] = month_sale.month_sale / month_sale.year_mean
    month_sale = month_sale.sort_values(by=['ratio', 'month_sale'], ascending=False)
    return list(month_sale.index[:top_n])


# Recomendations construction

In [23]:
def get_records_from_last_days(df, date, days=30):
    date_diff = (date - df.date).apply(lambda x: x.days)
    last_days_df = df[(date_diff <= days)]
    return last_days_df


def fill_recomendations(rec, products, limit):
    for prod in products:
        if prod not in rec:
            rec.append(prod)
            if len(rec) == limit:
                return


def make_personal_recomendation_list(df, customer, date):
    recomendations = []
    last_month_df = get_records_from_last_days(df, date)

    # 3 bestsellers from liked categories
    for cat in customer['cat']:
        recomendations.append(find_bestsellers(last_month_df, category=cat)[0])

    # 3 top products that similar people bought
    fill_recomendations(recomendations,
                        find_similar_customer_products(df, customer, top_n=6),
                        6)

    # 1 bestsellers from last month and the same state
    fill_recomendations(recomendations,
                        find_bestsellers(last_month_df,
                                         state=customer['loc']['state'],
                                         top_n=7),
                        7)

    # 2 trending product
    fill_recomendations(recomendations,
                        find_trending_products(last_month_df, date, top_n=9),
                        9)

    # 1 last month bestseller
    fill_recomendations(recomendations,
                        find_bestsellers(last_month_df, top_n=10),
                        10)

    return recomendations


def make_general_recomendation_list(df, date):
    recomendations = []
    last_month_df = get_records_from_last_days(df, date)
    top_categories = find_popular_categories(last_month_df, n=2)

    # 2 trending products from 2 most popular categories
    fill_recomendations(recomendations,
                        find_trending_products(df, date, top_n=2),
                        2)

    # 2 bestsellers from 2 most popular categories
    for limit, cat in enumerate(top_categories, start=3):
        fill_recomendations(recomendations,
                            find_bestsellers(last_month_df,
                                             category=cat,
                                             top_n=4),
                            limit)

    # 2 sezonal products
    fill_recomendations(recomendations,
                        find_sezonal_products(df, date.month, top_n=6),
                        6)

    # 2 last month bestsellers
    fill_recomendations(recomendations,
                        find_bestsellers(last_month_df, top_n=8),
                        8)

    # 2 absolute bestsellers
    fill_recomendations(recomendations,
                        find_bestsellers(df, top_n=10),
                        10)

    return recomendations


# Solution

In [24]:
make_personal_recomendation_list(complete_df, USER_1, DATE)

['fbce4c4cb307679d89a3bf3d3bb353b9',
 '002159fe700ed3521f46cfcf6e941c76',
 '7c1e2b3fa0233e46fb3bcdcb9919a72f',
 '704ef63ad0ac345842ea734060a83db2',
 '90ef6790cfc145cf45f9c476d1f47af0',
 '372645c7439f9661fbbacfd129aa92ec',
 '73326828aa5efe1ba096223de496f596',
 '027cdd14a677a5834bc67a9789db5021',
 'c6336fa91fbd87c359e44f5dca5a90ed',
 'e7cc48a9daff5436f63d3aad9426f28b']

In [25]:
make_personal_recomendation_list(complete_df, USER_2, DATE)

['054515fd15bc1a2029f10de97ffa9120',
 'e7cc48a9daff5436f63d3aad9426f28b',
 'b532349fe46b38fbc7bb3914c1bdae07',
 '35b9cce4c9cd4359551a3d702b25ad4c',
 'bbf920aa6ac72007a726e6f419156f8f',
 'cdc9d87e17ccf98f35634e6c3b77073e',
 'fbc1488c1a1e72ba175f53ab29a248e8',
 '027cdd14a677a5834bc67a9789db5021',
 'c6336fa91fbd87c359e44f5dca5a90ed',
 '73326828aa5efe1ba096223de496f596']

In [26]:
make_general_recomendation_list(complete_df, DATE)

['027cdd14a677a5834bc67a9789db5021',
 'c6336fa91fbd87c359e44f5dca5a90ed',
 'bb50f2e236e5eea0100680137654686c',
 'fbce4c4cb307679d89a3bf3d3bb353b9',
 '18b0e642cbae7251e60a64aa07dd9eb9',
 'c20a3f598c16d77249da67e81caa8317',
 'e7cc48a9daff5436f63d3aad9426f28b',
 '73326828aa5efe1ba096223de496f596',
 'aca2eb7d00ea1a7b8ebd4e68314663af',
 '99a4788cb24856965c36a24e339b6058']